<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Betting%20Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Create Baseline Data
Win_Prob = 0.5/(0.5+0.5)
Log_Loss = round(-np.log(1-(1-Win_Prob)),4)
Win_Prob
Log_Loss

0.6931

In [3]:
#Import Sportsbook data
nhl_lines_1920_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Odds%20Data/nhl%20odds%202019-20.csv')
nhl_lines_21_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Odds%20Data/nhl%20odds%202021.csv')
nhl_lines_2122_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Odds%20Data/nhl%20odds%202021-22.csv')

#nhl_lines_1920_raw.head()
#nhl_lines_21_raw.head()
#nhl_lines_2122_raw.head()

In [4]:
#Reduce data for CLV of Sportsbooks
nhl_lines_1920 = nhl_lines_1920_raw.loc[:,['Date','Rot','Team','Final','Close']]
nhl_lines_21 = nhl_lines_21_raw.loc[:,['Date','Rot','Team','Final','Close']]
nhl_lines_2122 = nhl_lines_2122_raw.loc[:,['Date','Rot','Team','Final','Close']]

nhl_lines_1920.head()
#nhl_lines_21.head()
#nhl_lines_2122.head()

,Date,Rot,Team,Final,Close
0,1002,1,Ottawa,3,270
1,1002,2,Toronto,5,-310
2,1002,3,Washington,3,135
3,1002,4,St.Louis,2,-150
4,1002,5,Vancouver,2,116


In [5]:
#Change Close column to float types
nhl_lines_1920.Close.astype(float)
nhl_lines_21.Close.astype(float)
nhl_lines_2122.Close.astype(float)

0       220.0
1      -250.0
2       205.0
3      -230.0
4      -102.0
        ...  
2797   -107.0
2798    170.0
2799   -190.0
2800   -109.0
2801   -101.0
Name: Close, Length: 2802, dtype: float64

In [6]:
#Make a column for the vig of all teams
nhl_lines_1920.loc[nhl_lines_1920['Close'] >= 0, 'Win %'] = 1/((nhl_lines_1920['Close']/100)+1)
nhl_lines_1920.loc[nhl_lines_1920['Close'] <= 0, 'Win %'] = 1/(1+(-100/nhl_lines_1920['Close']))

nhl_lines_21.loc[nhl_lines_21['Close'] >= 0, 'Win %'] = 1/((nhl_lines_21['Close']/100)+1)
nhl_lines_21.loc[nhl_lines_21['Close'] <= 0, 'Win %'] = 1/(1+(-100/nhl_lines_21['Close']))

nhl_lines_2122.loc[nhl_lines_2122['Close'] >= 0, 'Win %'] = 1/((nhl_lines_2122['Close']/100)+1)
nhl_lines_2122.loc[nhl_lines_2122['Close'] <= 0, 'Win %'] = 1/(1+(-100/nhl_lines_2122['Close']))

nhl_lines_1920

,Date,Rot,Team,Final,Close,Win %
0,1002,1,Ottawa,3,270,0.270270
1,1002,2,Toronto,5,-310,0.756098
2,1002,3,Washington,3,135,0.425532
3,1002,4,St.Louis,2,-150,0.600000
4,1002,5,Vancouver,2,116,0.462963
...,...,...,...,...,...,...
2419,925,28,Dallas,4,135,0.425532
2420,926,29,Dallas,3,150,0.400000
2421,926,30,TampaBay,2,-165,0.622642
2422,928,1,TampaBay,2,-170,0.629630


In [7]:
#Add column for Log Loss
nhl_lines_1920['Log Loss'] = -np.log(1-(1-nhl_lines_1920['Win %']))
nhl_lines_1920

,Date,Rot,Team,Final,Close,Win %,Log Loss
0,1002,1,Ottawa,3,270,0.270270,1.308333
1,1002,2,Toronto,5,-310,0.756098,0.279585
2,1002,3,Washington,3,135,0.425532,0.854415
3,1002,4,St.Louis,2,-150,0.600000,0.510826
4,1002,5,Vancouver,2,116,0.462963,0.770108
...,...,...,...,...,...,...,...
2419,925,28,Dallas,4,135,0.425532,0.854415
2420,926,29,Dallas,3,150,0.400000,0.916291
2421,926,30,TampaBay,2,-165,0.622642,0.473784
2422,928,1,TampaBay,2,-170,0.629630,0.462624


In [8]:
#Create table for each team
Senators = nhl_lines_1920.loc[nhl_lines_1920.Team=='Ottawa']
Senators

,Date,Rot,Team,Final,Close,Win %,Log Loss
0,1002,1,Ottawa,3,270,0.270270,1.308333
35,1005,52,Ottawa,1,118,0.458716,0.779325
101,1010,32,Ottawa,4,170,0.370370,0.993252
123,1012,60,Ottawa,4,230,0.303030,1.193922
159,1014,6,Ottawa,0,114,0.467290,0.760806
...,...,...,...,...,...,...,...
2036,303,19,Ottawa,3,260,0.277778,1.280934
2073,305,56,Ottawa,4,135,0.425532,0.854415
2108,307,15,Ottawa,2,140,0.416667,0.875469
2152,310,59,Ottawa,2,120,0.454545,0.788457


In [9]:
nhl_lines_1920.loc[nhl_lines_1920.index (%2 !=0)+1]

#nhl_lines_1920.loc[nhl_lines_1920.index %2 !=0, 'Win %'] = nhl_lines_1920['Vig']1/((nhl_lines_1920['Close']/100)+1)
#nhl_lines_1920.loc[nhl_lines_1920['Close'] <= 0, 'Vig'] = 1/(1+(-100/nhl_lines_1920['Close']))
#nhl_lines_1920

SyntaxError: ignored

**Stuff to Do Next - (have no more time, so quick thoughts that I have not checked, like the next one would be so easy to check if I had another 5 mins)**

Need to create a dataframe for each team. Use the loc function on player table first to get all the players for one team. Then do the same thing with goalies. We will probably need to keep them seperate tables since they do not have similar variables

Not necessary at this stage but eventually we are going to want to consolidate these code blocks. Ex, the code to create all of the team rosters should be in one block with no output. For now you can do 64 blocks (32 for skaters, 32 for goalies) and show output so we can see if there are any errors, but once we get past that stage we will comment out the output and put it all in one block. If we ever need to go back in check you just take out the '#' and run the code to see the output.

Need to add a column for 'status'. I am thinking the two states we use are 'active' and 'inactive' for the starting lineups. Will be 18 skaters and 1 goalie when we update daily but doesnt matter now. Should put this in before all of the loc functions so it shows up for the team dataframes. Note: will have to be added to both the full_player_totals dataframe and the goalie_stats dataframe.

Add folder in github for the csv's, update github links in code to pull data